In [1]:
# This was imported in order to run functions from other files
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [2]:
# This runs the data cleaning in ml_project
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Unique primary colors:
 1. black
 2. non-black
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
Unique primary colors:
 1. black
 2. non-black
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_1117490/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
Unique primary colors:
 1. black
 2. non-black
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [3]:
#kog_reg
nb = nbformat.read("log_reg.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "log_reg_normalized.ipynb")

# Random Frest
nb = nbformat.read("random_forest.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "random_forest_normalized.ipynb")

# X Random Trees
nb = nbformat.read("x_random_trees.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "x_random_trees_normalized.ipynb")

# XGBoost
nb = nbformat.read("xg_boost.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "xg_boost_normalized.ipynb")

# Multiclass OVR XG
nb = nbformat.read("ovr_xg_boost.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "ovr_xg_boost_normalized.ipynb")

# LightGBM
nb = nbformat.read("lightgbm.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "lightgbm_normalized.ipynb")

In [4]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline  # Important for SMOTE compatibility
from imblearn.over_sampling import SMOTENC

In [ ]:
'''
Whenever I changed something in these files, I would uncomment something to run it briefly, then comment it 
out again. When you run for the FIRST time, pls run each comment line by line and comment one line at a time
'''


#%run log_reg.ipynb
#%run random_forest.ipynb
#%run x_random_trees.ipynb
#%run xg_boost.ipynb
#%run ovr_xg_boost.ipynb
#%run lightgbm.ipynb
#df_train.head()

In [ ]:
# old version of training classifier, no longer in use
def train_voting_classifier(X_train, y_train, X_test, models, voting='soft'):
    """
    Trains a VotingClassifier with the given models and evaluates it on the test data.
    
    Parameters:
    - X_train: Training features
    - y_train: Training labels
    - X_test: Test features
    - models: List of tuples (model_name, model_instance)
    - voting: 'hard' or 'soft' for the type of voting to be used (default is 'soft')
    
    Returns:
    - trained_model: Trained VotingClassifier
    - test_predictions: Predictions on the test set
    """
    # Initialize the VotingClassifier with the given models
    voting_clf = VotingClassifier(estimators=models, voting=voting, n_jobs=-1)
    
    # Fit the model
    voting_clf.fit(X_train, y_train) #   voting_clf.fit_transform(X_test,)
    
    # Make predictions
    test_predictions = voting_clf.predict(X_test)
    
    # Evaluate the model
    accuracy = balanced_accuracy_score(y_train, voting_clf.predict(X_train))
    print(f"Training Accuracy: {accuracy:.4f}")
    
    return voting_clf, test_predictions


In [ ]:
# Additional data cleaning to try out different features that might have worked better
df_train = bucket_seasons(df_train)
df_test = bucket_seasons(df_test)

df_train = bucket_days(df_train)
df_test = bucket_days(df_test)

df_train['breed'] = df_train['breed'].astype(str)
df_test['breed'] = df_test['breed'].astype(str)

df_train = df_train.drop(columns=['size', 'intake_hour', 'intake_month'])
df_test = df_test.drop(columns=['size', 'intake_hour', 'intake_month'])



# encodes breed to stop errors
def encode_breed(val):
    if isinstance(val, int):
        return val  # Keep numeric as-is
    mapping = {'0' : 0, '1' : 1, '2' : 2, '3' : 3, '4' : 4, '5' : 5, '6' : 6, '7' : 7, '8' : 8, 'Common': 9, 'Rare': 10, 'Unknown': 11}
    return mapping.get(val, -1)  # Return -1 for unexpected values (or handle as needed)

# Apply transformation
df_train['breed'] = df_train['breed'].apply(encode_breed)
df_test['breed'] = df_test['breed'].apply(encode_breed)



df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,outcome_type,season,time_of_day
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,non-black,0,Return to Owner,Summer,Afternoon
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,non-black,1,Return to Owner,Spring,Evening
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,9,2022,non-black,0,Transfer,Spring,Night
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,non-black,1,Return to Owner,Winter,Afternoon
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,Return to Owner,Spring,Morning


In [ ]:
df_train['breed'].value_counts()

breed
9     47258
4     13754
0     13398
2      9803
5      9663
3      6505
7      4240
10     2569
8      2163
1      1785
6        15
11        2
Name: count, dtype: int64

In [ ]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [ ]:
# This was an attempt to use Voting Classifier method but too many errors happened so i stopped using it

cat_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed','season', 'is_mix', 'time_of_day', 'primary_color'] # experimenting with droping color
num_cols = ['age_upon_intake', 'intake_year']    # Replace with your actual numerical columns
# freq_cols = ['primary_color'] # for trees



'''
# This is just used to get the model with the best classifiers
models = [
    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression
    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest
    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees
]

# Assuming X_train, y_train, X_test are already defined in your environment
voting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')

classification_report_with_accuracy_score(y_test, test_predictions)
'''

"\n# This is just used to get the model with the best classifiers\nmodels = [\n    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression\n    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest\n    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees\n]\n\n# Assuming X_train, y_train, X_test are already defined in your environment\nvoting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')\n\nclassification_report_with_accuracy_score(y_test, test_predictions)\n"

In [ ]:
X_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,season,time_of_day
107615,Stray,Normal / Behavior,Cat,Intact Female,1.0,9,2021,non-black,0,Summer,Morning
13002,Stray,Injured,Dog,Intact Male,24.0,3,2014,non-black,1,Summer,Morning
32513,Public Assist,Normal / Behavior,Dog,Intact Female,12.0,4,2019,non-black,1,Summer,Night
79497,Stray,Normal / Behavior,Cat,Intact Female,1.0,9,2015,non-black,1,Summer,Evening
88489,Stray,Normal / Behavior,Dog,Intact Female,4.0,2,2017,black,0,Spring,Morning


In [ ]:
# used for extra valdiation to find the best thresholds to maximize recall per class
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [ ]:
# this was used for cross validation so that I could find the best thresholds to use for voting so that I penalize
# models for over guessing 'Adopted'

'''

X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Train on sub-training set
logreg_CV = train_logreg_classifier(X_train_sub, y_train_sub, X_val, cat_cols, num_cols)[0]
rf_CV = train_RF_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]
xrt_CV = train_XRT_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]
xg = train_XG_classifier(X_train_sub, y_train_sub, X_val, [], cat_cols)[0]

# Get predicted probabilities on validation set
log_probs_CV = logreg_CV.predict_proba(X_test)
rf_probs_CV = rf_CV.predict_proba(X_test)
xrt_probs_CV = xrt_CV.predict_proba(X_test)
xg_probs_CV = xg.predict_proba(X_test)

# Average the probabilities
avg_probs_val = (log_probs_CV + rf_probs_CV + xrt_probs_CV + xg_probs_CV) / 4


from itertools import product

# Define classes (adjust as needed)
classes = [0, 1, 2, 3, 4]

# Threshold options for each class
threshold_grid = {
    0: [0.5, 0.6, 0.7, 0.8],
    1: [0.05, 0.1, 0.2],
    2: [0.1, 0.2, 0.3],
    3: [0.2, 0.3, 0.4],
    4: [0.2, 0.3, 0.4]
}

# Generate all threshold combinations
all_thresholds = list(product(*threshold_grid.values()))

# Tune thresholds
best_score = -1
best_thresholds = None

for thresh_combo in all_thresholds:
    thresholds = dict(zip(classes, thresh_combo))
    preds = []

    for probs in avg_probs_val:
        passed = [cls for cls, prob in enumerate(probs) if prob >= thresholds.get(cls, 1.0)]
        if passed:
            chosen = max(passed, key=lambda cls: probs[cls])
        else:
            chosen = np.argmax(probs)
        preds.append(chosen)

    score = balanced_accuracy_score(y_val, preds)

    if score > best_score:
        best_score = score
        best_thresholds = thresholds

print("Best thresholds:", best_thresholds)
print("Best balanced accuracy on validation:", best_score)
'''

'\n\nX_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)\n\n# Train on sub-training set\nlogreg_CV = train_logreg_classifier(X_train_sub, y_train_sub, X_val, cat_cols, num_cols)[0]\nrf_CV = train_RF_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]\nxrt_CV = train_XRT_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]\nxg = train_XG_classifier(X_train_sub, y_train_sub, X_val, [], cat_cols)[0]\n\n# Get predicted probabilities on validation set\nlog_probs_CV = logreg_CV.predict_proba(X_test)\nrf_probs_CV = rf_CV.predict_proba(X_test)\nxrt_probs_CV = xrt_CV.predict_proba(X_test)\nxg_probs_CV = xg.predict_proba(X_test)\n\n# Average the probabilities\navg_probs_val = (log_probs_CV + rf_probs_CV + xrt_probs_CV + xg_probs_CV) / 4\n\n\nfrom itertools import product\n\n# Define classes (adjust as needed)\nclasses = [0, 1, 2, 3, 4]\n\n# Threshold options for each class\nthreshold_grid = {\n    0: [0.5, 0.6, 0.

In [ ]:
df_train.head()


,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,outcome_type,season,time_of_day
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,non-black,0,Return to Owner,Summer,Afternoon
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,non-black,1,Return to Owner,Spring,Evening
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,9,2022,non-black,0,Transfer,Spring,Night
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,non-black,1,Return to Owner,Winter,Afternoon
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,Return to Owner,Spring,Morning


In [ ]:
custom_thresholds = {
    0: 0.6,
    1: 0.5,
    2: 0.5,
    3: 0.5,
    4: 0.5
}

In [ ]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# manual voting - VotingClassifer was broken and with manual voting \
# This is an implementaion of soft voting that gets the probability of a record belong to a class for each model

#def manual_voting(models):
logreg = train_logreg_classifier(X_train, y_train, df_test, cat_cols, num_cols)[0] #logistic regression
rf = train_RF_classifier(X_train, y_train, df_test, cat_cols)[0] #random forest
xrt = train_XRT_classifier(X_train, y_train, df_test, cat_cols)[0] #x random trees
xg = train_XG_classifier(X_train,y_train, df_test, [], cat_cols)[0]
lgbm = train_classifier_lightgbm(X_train, y_train, X_test, [], cat_cols, custom_thresholds=custom_thresholds)



log_probs = logreg.predict_proba(X_test)
rf_probs = rf.predict_proba(X_test)
xrt_probs = xrt.predict_proba(X_test)
xg_probs = xg.predict_proba(X_test)
lgbm_probs = lgbm.predict_proba(X_test)
#ovr_probs = xg.predict_proba(X_test)

avg_probs = (log_probs + rf_probs + xrt_probs + xg_probs + lgbm_probs ) / 5


# potential weighted average where some models get a bigger voice in determine a record
'''
avg_probs = (
    0.1667 * log_probs +
    0.1667 * rf_probs +
    0.1667 * xrt_probs +
    0.5    * xg_progs
) 

'''

het_thresholds = {
    0: 0.60,  # Adopted
    1: 0.20,  # Died
    2: 0.30,  # Euth
    3: 0.40,  # RTO
    4: 0.30   # Transfer
}

#le.fit_transform(y_train)

# picks the class with the highest average probability among the classes that meet the threshold
custom_preds = []

for probs in avg_probs:
    # Find which classes exceed their thresholds
    passed = [cls for cls, prob in enumerate(probs) if prob >= het_thresholds.get(cls, 1.0)]
    
    if passed:
        # Among the passed, pick the class with the highest probability
        chosen = max(passed, key=lambda cls: probs[cls])
    else:
        # Default to the class with highest average prob
        chosen = np.argmax(probs)
        
    custom_preds.append(chosen)

test_preds = np.array(custom_preds)
test_preds_labels = le.inverse_transform(custom_preds)

classification_report_with_accuracy_score(y_test, test_preds_labels)
#save_predictions(test_preds_labels, 'voting')

NameError: name 'train_logreg_classifier' is not defined